In [1]:
#Import Packages
import pandas as pd
import numpy as np
import os
import itertools
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import datetime
import matplotlib
from herbie import Herbie
import pickle
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import pysolar.solar as solar
from geographiclib.geodesic import Geodesic
import xarray as xr
import pytz
import simplekml
import cartopy
import sys
sys.path.append('..')
import funcs.ac_funcs as ac
import funcs.plotting as plotting


pd.options.mode.chained_assignment = None


In [16]:
base_project_dir = '/Users/agmeyer4/LAIR_1/'

#define datetime range and interval parameters
dt1_str = '2023-07-10 00:00:00'#start datetime
dt2_str = '2023-07-10 23:59:59' #end datetime
timezone = 'UTC' #this should nearly always be UTC, but can identify if your datetime strings above are in a different timezone
dt1 = ac.dtstr_to_dttz(dt1_str,timezone)
dt2 = ac.dtstr_to_dttz(dt2_str,timezone)

oof_data_folder_ha = os.path.join(base_project_dir,'Data/EM27_oof/ha')
my_oof_manager_ha = ac.oof_manager(oof_data_folder_ha,timezone)
oof_df_ha = my_oof_manager_ha.load_oof_df_inrange(dt1,dt2)
oof_df_ha = oof_df_ha.loc[oof_df_ha['flag']==0]
oof_df_ha = oof_df_ha.resample('5T').mean()

oof_data_folder_ua = os.path.join(base_project_dir,'Data/EM27_oof/ua')
my_oof_manager_ua= ac.oof_manager(oof_data_folder_ua,timezone)
oof_df_ua = my_oof_manager_ua.load_oof_df_inrange(dt1,dt2)
oof_df_ua = oof_df_ua.loc[oof_df_ua['flag']==0]
oof_df_ua = oof_df_ua.resample('5T').mean()



/var/folders/sn/n0kw7krj4753ps5l21g6r9zr0000gn/T/ipykernel_63776/1828026377.py:14: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/sn/n0kw7krj4753ps5l21g6r9zr0000gn/T/ipykernel_63776/1828026377.py:20: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [17]:
# xlims= [datetime.datetime.combine(em27_resampled.index[0].date(),datetime.time(hour=6)),
#         datetime.datetime.combine(em27_resampled.index[-1].date(),datetime.time(hour=22))]
spec = 'xch4(ppm)'
fig = go.Figure()
fig.add_trace(go.Scatter(
        x = oof_df_ha.index,
        y = oof_df_ha[spec],
        name='ha',
        # error_y = dict(
        #     type='data', # value of error bar given in data coordinates
        #     array=oof_df_cindy[f'{spec}_error'],
        #     visible=True,
        #     color = 'grey',
        #     width = 0.8),
        mode='markers',
        marker={'color':'black'},
    ))
fig.add_trace(go.Scatter(
        x = oof_df_ua.index,
        y = oof_df_ua[spec],
        name='ua',
        # error_y = dict(
        #     type='data', # value of error bar given in data coordinates
        #     array=oof_df_agm[f'{spec}_error'],
        #     visible=True,
        #     color = 'pink',
        #     width = 0.1),
        mode='markers',
        marker={'color':'red','size':2},
    ))
fig.update_yaxes(title_text = spec)



fig.update_layout(
    height=400,
)
fig.show()